# Continuous-Time Markov Chains: The Yule-Furry Process

In this notebook, we will simulate a continuous-time Markov chain (CTMC) and 
compare the results to known analytical formulas. The example we will use is 
the **Yule-Furry process**, also known as the linear birth process.

Recall from the board that in this process, each individual independently gives 
birth at rate $\beta$. When the population size is $n$, the total birth rate is 
$n\beta$. Since there are no deaths, the population can only grow.

## Why are inter-event times exponentially distributed?

The defining property of a CTMC is that it is *memoryless*: the future depends 
only on the current state, not on how long the system has been in that state. 
The exponential distribution is the unique continuous distribution with this 
memoryless property — if $T \sim \text{Exp}(\lambda)$, then 
$P(T > t + s \mid T > t) = P(T > s)$ for all $t, s \geq 0$.

For a single individual with birth rate $\beta$, the time until its next birth 
is $\text{Exp}(\beta)$. When there are $n$ independent individuals each with 
rate $\beta$, the time until the *first* birth among all of them is the minimum 
of $n$ independent exponentials. A standard result from probability is that the 
minimum of $n$ independent $\text{Exp}(\beta)$ random variables is 
$\text{Exp}(n\beta)$. So when the population is $n$, the time until the next 
event is exponentially distributed with rate $n\beta$ (equivalently, mean 
$1/(n\beta)$).

## The Gillespie algorithm

The **Gillespie algorithm** (also called the stochastic simulation algorithm) is 
a general method for exactly simulating CTMCs. The idea is simple: rather than 
advancing time in fixed steps and checking whether events occur (which would 
require very small time steps for accuracy), we jump directly from one event to 
the next. At each step:
1. Compute the total rate of all possible events given the current state.
2. Draw the time until the next event from the corresponding exponential 
distribution.
3. Determine which event occurs (in our case there is only one type — a birth).
4. Update the state and repeat.

This is exact (no discretization error) and efficient, since we skip over all 
the dead time between events. For the Yule-Furry process, the algorithm is 
particularly simple because there is only one type of event (birth), so step 3 
is trivial.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import comb # computes binomial coefficients, C(n, k)

## Simulating a single realization

The simulation algorithm is as follows:
1. Start with $n_0$ individuals at time $t=0$.
2. Draw an inter-event time $\Delta t$ from $\text{Exp}(n\beta)$. In numpy, 
`rng.exponential(scale)` takes the *scale* parameter, which is $1/\text{rate} = 1/(n\beta)$.
3. Advance time: $t \leftarrow t + \Delta t$.
4. If $t > T_{\max}$, stop.
5. Otherwise, increment the population: $n \leftarrow n + 1$.
6. Record the new time and population size, and go back to step 2.

In the cell below, implement this algorithm using a while loop. Store the 
times and population sizes in lists so that you can plot them afterward. Use 
`plt.step()` to plot the result as a step function (which is the correct 
visualization for a jump process).

In [ ]:
rng = np.random.default_rng()

# Parameters
beta = 0.1
n0 = 5
T_max = 10

# Initialize lists to store the trajectory
times = [0.0]
pop = [n0]

# Run the simulation
t = 0.0
n = n0
while True:
    # Draw inter-event time
    dt = rng.exponential(1.0 / (n * beta))
    t += dt
    if t > T_max:
        break
    n += 1
    times.append(t)
    pop.append(n)

# Append the final time so the plot extends to T_max
times.append(T_max)
pop.append(n)

# TODO: Plot the trajectory using plt.step(times, pop, where='post')


## Running many simulations

A single realization tells us little about the statistics of the process. To 
understand the distribution of outcomes, we need to run many simulations and 
collect the results.

First, wrap your simulation code from above into a function that takes 
parameters and returns the trajectory (times and population sizes). Then, run 
it `N_sim` times and:
1. Plot all trajectories on the same axes.
2. Overlay the analytical expected value $E[N(t)] = n_0 e^{\beta t}$ as a 
dashed line for comparison.

In [ ]:
def simulate_yule_furry(beta, n0, T_max, rng):
    """Simulate one realization of the Yule-Furry process.
    
    Returns lists of times and population sizes."""
    times = [0.0]
    pop = [n0]
    # TODO: Fill in the simulation loop (same as above)

    return times, pop

# Parameters
beta = 0.1
n0 = 5
T_max = 10
N_sim = 200

rng = np.random.default_rng()

# TODO: Run N_sim simulations. For each one:
#   - Call simulate_yule_furry and store the final population size.
#   - Plot the trajectory with plt.step(..., where='post', alpha=0.15, color='blue', linewidth=0.5)
#     so the trajectories are semi-transparent.
final_pops = np.zeros(N_sim, dtype=int)


# Overlay the analytical expected value
# t_analytical = np.linspace(0, T_max, 200)
# E_N = n0 * np.exp(beta * t_analytical)
# plt.plot(t_analytical, E_N, 'r--', linewidth=2, label='$E[N(t)] = n_0 e^{\\beta t}$')

# TODO: Print the sample mean and analytical mean n0*exp(beta*T_max) to compare.
#   You can also compare the sample variance to the analytical variance:
#   Var[N(t)] = n0 * (1 - exp(-beta*t)) * exp(2*beta*t)


## Comparing to the analytical distribution

The Yule-Furry process has a known probability distribution. For $n \geq n_0$,
$$
p_n(t) = \binom{n-1}{n_0-1} e^{-\beta n_0 t} \left(1 - e^{-\beta t}\right)^{n - n_0}.
$$
This is a *negative binomial distribution* in which the probability of 
"success" in a single trial, $e^{-\beta t}$, decreases exponentially with time.

In the cell below, create a histogram of the final population sizes from your 
simulations and overlay the analytical PMF. Use `density=True` in `plt.hist()` 
so that the histogram is normalized to a probability distribution. To compute 
the binomial coefficient, use `scipy.special.comb` which we imported above.

In [ ]:
# Compute the analytical PMF over a range of population sizes
n_vals = np.arange(n0, np.max(final_pops) + 5)
p_analytical = comb(n_vals - 1, n0 - 1) * \
    np.exp(-beta * n0 * T_max) * (1 - np.exp(-beta * T_max))**(n_vals - n0)

# TODO: Plot a histogram of final_pops and overlay p_analytical.
#   - Use integer-width bins: np.arange(n0 - 0.5, np.max(final_pops) + 1.5, 1)
#   - Use density=True so the histogram is normalized.
#   - Plot the analytical PMF with plt.plot(n_vals, p_analytical, 'ro-', markersize=4)


## Further exploration

Try experimenting with different parameter values:
- What happens when you increase $\beta$? The population grows faster, leading 
to larger final sizes and wider distributions. Be careful with large $\beta$ or 
large $T_{\max}$ as the population can grow very large and the simulation will 
slow down.
- What happens when $n_0 = 1$? The distribution becomes a geometric 
distribution. Try it!
- How many simulations do you need for the histogram to closely match the 
analytical PMF?